## Data Wrangling: Clean, Transform, Merge, Reshape

In [1]:
import pandas as pd

## Combining and merging data sets

### Database-style DataFrame merges

In [2]:
df1 = pd.DataFrame({'data1' : range(5,12), 'key' : list('bbacaab')})
df2 = pd.DataFrame({'data2' : range(56,59), 'key' : list('abd')})

In [9]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [10]:
df2

,data2,key
0,56,a
1,57,b
2,58,d


In [7]:
df1.merge(df2)
# devuelve aquellas para las que ha encontrado una correspondencia, es el inner join
# outer join devuelve todas tengan correspondencia o no

,data1,key,data2
0,5,b,57
1,6,b,57
2,11,b,57
3,7,a,56
4,9,a,56
5,10,a,56


In [12]:
df1.merge(df2, how='outer')

,data1,key,data2
0,5.0,b,57.0
1,6.0,b,57.0
2,11.0,b,57.0
3,7.0,a,56.0
4,9.0,a,56.0
5,10.0,a,56.0
6,8.0,c,NaN
7,NaN,d,58.0


By default, .merge() performs an [inner join](https://www.w3schools.com/sql/sql_join.asp) between the DataFrames, using the common columns as keys.

That means that it returns the cartesian product of the elements with common keys: if there are duplicates, it will return all the possible combinations:

In [4]:
df3 = pd.DataFrame({'data3' : range(56,61), 'key' : list('abdbd')})
df1.merge(df3)

,data1,key,data3
0,5,b,57
1,5,b,59
2,6,b,57
3,6,b,59
4,11,b,57
5,11,b,59
6,7,a,56
7,9,a,56
8,10,a,56


If the columns to join on don't have the same name, or we want to join on the index of the DataFrames, we'll need to specify that.

In [5]:
df4 = pd.DataFrame({'data4' : range(56,61), 'rkey' : list('abdbd')})
df3.merge(df4)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [8]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [9]:
df4

,data4,rkey
0,56,a
1,57,b
2,58,d
3,59,b
4,60,d


In [15]:
df1.merge(df4, left_on='key', right_on='rkey')

,data1,key,data4,rkey
0,5,b,57,b
1,5,b,59,b
2,6,b,57,b
3,6,b,59,b
4,11,b,57,b
5,11,b,59,b
6,7,a,56,a
7,9,a,56,a
8,10,a,56,a


If there are two columns with the same name that we do not join on, both will get transferred to the resulting DataFrame with a suffix. We can customize these suffixes.

In [10]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [11]:
df2

,data2,key
0,56,a
1,57,b
2,58,d


In [17]:
df1.merge(df2, left_on='data1', right_on='data2', how='outer')

,data1,key_x,data2,key_y
0,5.0,b,NaN,NaN
1,6.0,b,NaN,NaN
2,7.0,a,NaN,NaN
3,8.0,c,NaN,NaN
4,9.0,a,NaN,NaN
5,10.0,a,NaN,NaN
6,11.0,b,NaN,NaN
7,NaN,NaN,56.0,a
8,NaN,NaN,57.0,b
9,NaN,NaN,58.0,d


In [18]:
df1.merge(df2, left_on='data1', right_on='data2', how='outer', suffixes=['_costumer','_order'])

,data1,key_costumer,data2,key_order
0,5.0,b,NaN,NaN
1,6.0,b,NaN,NaN
2,7.0,a,NaN,NaN
3,8.0,c,NaN,NaN
4,9.0,a,NaN,NaN
5,10.0,a,NaN,NaN
6,11.0,b,NaN,NaN
7,NaN,NaN,56.0,a
8,NaN,NaN,57.0,b
9,NaN,NaN,58.0,d


### Merging on index

In [20]:
df4.index = range(5,10)
df4

,data4,rkey
5,56,a
6,57,b
7,58,d
8,59,b
9,60,d


In [21]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [22]:
df1.merge(df4, left_on='data1', right_index=True)

,data1,key,data4,rkey
0,5,b,56,a
1,6,b,57,b
2,7,a,58,d
3,8,c,59,b
4,9,a,60,d


### Concatenating along an axis

In [23]:
pd.concat([df1, df2])
# los une uno debajo de otro, por defecto

/home/elsa/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,data1,data2,key
0,5.0,NaN,b
1,6.0,NaN,b
2,7.0,NaN,a
3,8.0,NaN,c
4,9.0,NaN,a
5,10.0,NaN,a
6,11.0,NaN,b
0,NaN,56.0,a
1,NaN,57.0,b
2,NaN,58.0,d


In [24]:
pd.concat([df1, df2], axis=1)
# los pega ahi porque tienen el mismo index, no porque sean los primeros

,data1,key,data2,key
0,5,b,56.0,a
1,6,b,57.0,b
2,7,a,58.0,d
3,8,c,NaN,NaN
4,9,a,NaN,NaN
5,10,a,NaN,NaN
6,11,b,NaN,NaN


In [25]:
pd.concat([df1, df4], axis=1)

,data1,key,data4,rkey
0,5.0,b,NaN,NaN
1,6.0,b,NaN,NaN
2,7.0,a,NaN,NaN
3,8.0,c,NaN,NaN
4,9.0,a,NaN,NaN
5,10.0,a,56.0,a
6,11.0,b,57.0,b
7,NaN,NaN,58.0,d
8,NaN,NaN,59.0,b
9,NaN,NaN,60.0,d


In [27]:
pd.concat([df1['data1'], df2['data2'], df3['data3']])

0     5
1     6
2     7
3     8
4     9
5    10
6    11
0    56
1    57
2    58
0    56
1    57
2    58
3    59
4    60
dtype: int64

In [29]:
pd.concat([df1['data1'], df2['data2'], df3['data3']], axis=1, keys=['x','y','z'])

,x,y,z
0,5,56.0,56.0
1,6,57.0,57.0
2,7,58.0,58.0
3,8,NaN,59.0
4,9,NaN,60.0
5,10,NaN,NaN
6,11,NaN,NaN


#### Digression

Attention! Be careful not to reassign to reserved words or functions- you will overwrite the variable.

In [ ]:
#pd.concat = [df1, df2]
pdf.concat([df3,df4])

You can delete the overwritten variable, but you won't get back the original value. If it is an object or function from a module, you'll need to reload() the module, since Python doesn't load again an already imported module if you try to import it. reload() is useful also when you are actively developing your own module and want to load the latest definition of a function into memory.

In [ ]:
# from importlib import reload
# reload(pd)
# pd.concat([df3,df4])

## Data transformation

### Removing duplicates

In [31]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [30]:
df1.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6    False
dtype: bool

In [34]:
df1['key'].duplicated()

0    False
1     True
2    False
3    False
4     True
5     True
6     True
Name: key, dtype: bool

In [35]:
df1['key'].drop_duplicates()

0    b
2    a
3    c
Name: key, dtype: object

In [36]:
df1.drop_duplicates(subset='key')

,data1,key
0,5,b
2,7,a
3,8,c


In [37]:
df1.drop_duplicates(subset='key', keep='last')

,data1,key
3,8,c
5,10,a
6,11,b


In [38]:
df1.drop_duplicates(subset='key', keep='first')

,data1,key
0,5,b
2,7,a
3,8,c


### Renaming axis indexes

In [39]:
df1

,data1,key
0,5,b
1,6,b
2,7,a
3,8,c
4,9,a
5,10,a
6,11,b


In [40]:
df1.index = list('bhkdsdf')
df1

,data1,key
b,5,b
h,6,b
k,7,a
d,8,c
s,9,a
d,10,a
f,11,b


### Discretization and binning

In [42]:
import numpy as np

np.random.seed(42)
ages = pd.Series(np.random.randint(9,99,50))
ages

0     60
1     23
2     80
3     69
4     29
5     91
6     95
7     83
8     83
9     96
10    32
11    11
12    30
13    61
14    10
15    96
16    38
17    46
18    10
19    72
20    68
21    29
22    41
23    84
24    66
25    30
26    97
27    57
28    67
29    50
30    68
31    88
32    23
33    70
34    70
35    55
36    70
37    59
38    63
39    72
40    11
41    59
42    15
43    29
44    81
45    47
46    26
47    12
48    97
49    68
dtype: int64

In [43]:
limits = [14, 18, 35, 50, 65]

pd.cut(ages, limits)

0     (50.0, 65.0]
1     (18.0, 35.0]
2              NaN
3              NaN
4     (18.0, 35.0]
5              NaN
6              NaN
7              NaN
8              NaN
9              NaN
10    (18.0, 35.0]
11             NaN
12    (18.0, 35.0]
13    (50.0, 65.0]
14             NaN
15             NaN
16    (35.0, 50.0]
17    (35.0, 50.0]
18             NaN
19             NaN
20             NaN
21    (18.0, 35.0]
22    (35.0, 50.0]
23             NaN
24             NaN
25    (18.0, 35.0]
26             NaN
27    (50.0, 65.0]
28             NaN
29    (35.0, 50.0]
30             NaN
31             NaN
32    (18.0, 35.0]
33             NaN
34             NaN
35    (50.0, 65.0]
36             NaN
37    (50.0, 65.0]
38    (50.0, 65.0]
39             NaN
40             NaN
41    (50.0, 65.0]
42    (14.0, 18.0]
43    (18.0, 35.0]
44             NaN
45    (35.0, 50.0]
46    (18.0, 35.0]
47             NaN
48             NaN
49             NaN
dtype: category
Categories (4, interval[int64]): [

In [46]:
categorized = pd.cut(ages, limits, include_lowest=True)
categorized

0       (50.0, 65.0]
1       (18.0, 35.0]
2                NaN
3                NaN
4       (18.0, 35.0]
5                NaN
6                NaN
7                NaN
8                NaN
9                NaN
10      (18.0, 35.0]
11               NaN
12      (18.0, 35.0]
13      (50.0, 65.0]
14               NaN
15               NaN
16      (35.0, 50.0]
17      (35.0, 50.0]
18               NaN
19               NaN
20               NaN
21      (18.0, 35.0]
22      (35.0, 50.0]
23               NaN
24               NaN
25      (18.0, 35.0]
26               NaN
27      (50.0, 65.0]
28               NaN
29      (35.0, 50.0]
30               NaN
31               NaN
32      (18.0, 35.0]
33               NaN
34               NaN
35      (50.0, 65.0]
36               NaN
37      (50.0, 65.0]
38      (50.0, 65.0]
39               NaN
40               NaN
41      (50.0, 65.0]
42    (13.999, 18.0]
43      (18.0, 35.0]
44               NaN
45      (35.0, 50.0]
46      (18.0, 35.0]
47           

In [47]:
ages[categorized.isna()]

2     80
3     69
5     91
6     95
7     83
8     83
9     96
11    11
14    10
15    96
18    10
19    72
20    68
23    84
24    66
26    97
28    67
30    68
31    88
33    70
34    70
36    70
39    72
40    11
44    81
47    12
48    97
49    68
dtype: int64

In [48]:
limits = [0, 14, 18, 35, 50, 65, 100]

pd.cut(ages, limits)

0      (50, 65]
1      (18, 35]
2     (65, 100]
3     (65, 100]
4      (18, 35]
5     (65, 100]
6     (65, 100]
7     (65, 100]
8     (65, 100]
9     (65, 100]
10     (18, 35]
11      (0, 14]
12     (18, 35]
13     (50, 65]
14      (0, 14]
15    (65, 100]
16     (35, 50]
17     (35, 50]
18      (0, 14]
19    (65, 100]
20    (65, 100]
21     (18, 35]
22     (35, 50]
23    (65, 100]
24    (65, 100]
25     (18, 35]
26    (65, 100]
27     (50, 65]
28    (65, 100]
29     (35, 50]
30    (65, 100]
31    (65, 100]
32     (18, 35]
33    (65, 100]
34    (65, 100]
35     (50, 65]
36    (65, 100]
37     (50, 65]
38     (50, 65]
39    (65, 100]
40      (0, 14]
41     (50, 65]
42     (14, 18]
43     (18, 35]
44    (65, 100]
45     (35, 50]
46     (18, 35]
47      (0, 14]
48    (65, 100]
49    (65, 100]
dtype: category
Categories (6, interval[int64]): [(0, 14] < (14, 18] < (18, 35] < (35, 50] < (50, 65] < (65, 100]]

In [49]:
limits = [0, 14, 18, 35, 50, 65, 100]

pd.cut(ages, limits).value_counts()

(65, 100]    23
(18, 35]      9
(50, 65]      7
(35, 50]      5
(0, 14]       5
(14, 18]      1
dtype: int64

## String manipulation

### String object methods

In [53]:
bichos = pd.Series(np.random.choice(['Mantis Shrimp', 'Naked Mole Rat', 'Star Nosed Mole'], 15))
bichos

0     Star Nosed Mole
1       Mantis Shrimp
2       Mantis Shrimp
3       Mantis Shrimp
4     Star Nosed Mole
5       Mantis Shrimp
6       Mantis Shrimp
7     Star Nosed Mole
8     Star Nosed Mole
9     Star Nosed Mole
10      Mantis Shrimp
11    Star Nosed Mole
12    Star Nosed Mole
13      Mantis Shrimp
14    Star Nosed Mole
dtype: object

In [55]:
bichos.str.upper()

0     STAR NOSED MOLE
1       MANTIS SHRIMP
2       MANTIS SHRIMP
3       MANTIS SHRIMP
4     STAR NOSED MOLE
5       MANTIS SHRIMP
6       MANTIS SHRIMP
7     STAR NOSED MOLE
8     STAR NOSED MOLE
9     STAR NOSED MOLE
10      MANTIS SHRIMP
11    STAR NOSED MOLE
12    STAR NOSED MOLE
13      MANTIS SHRIMP
14    STAR NOSED MOLE
dtype: object

In [56]:
bichos.str.lower()

0     star nosed mole
1       mantis shrimp
2       mantis shrimp
3       mantis shrimp
4     star nosed mole
5       mantis shrimp
6       mantis shrimp
7     star nosed mole
8     star nosed mole
9     star nosed mole
10      mantis shrimp
11    star nosed mole
12    star nosed mole
13      mantis shrimp
14    star nosed mole
dtype: object

In [57]:
bichos.str.len()

0     15
1     13
2     13
3     13
4     15
5     13
6     13
7     15
8     15
9     15
10    13
11    15
12    15
13    13
14    15
dtype: int64

In [58]:
bichos.str.split()

0     [Star, Nosed, Mole]
1        [Mantis, Shrimp]
2        [Mantis, Shrimp]
3        [Mantis, Shrimp]
4     [Star, Nosed, Mole]
5        [Mantis, Shrimp]
6        [Mantis, Shrimp]
7     [Star, Nosed, Mole]
8     [Star, Nosed, Mole]
9     [Star, Nosed, Mole]
10       [Mantis, Shrimp]
11    [Star, Nosed, Mole]
12    [Star, Nosed, Mole]
13       [Mantis, Shrimp]
14    [Star, Nosed, Mole]
dtype: object

In [59]:
bichos.str[:6]

0     Star N
1     Mantis
2     Mantis
3     Mantis
4     Star N
5     Mantis
6     Mantis
7     Star N
8     Star N
9     Star N
10    Mantis
11    Star N
12    Star N
13    Mantis
14    Star N
dtype: object

In [61]:
bichos.str.split().str[-1]

0       Mole
1     Shrimp
2     Shrimp
3     Shrimp
4       Mole
5     Shrimp
6     Shrimp
7       Mole
8       Mole
9       Mole
10    Shrimp
11      Mole
12      Mole
13    Shrimp
14      Mole
dtype: object

### Vectorized string functions in pandas

[Vectorized string functions in pandas](https://pandas.pydata.org/pandas-docs/stable/text.html) are grouped within the .str attribute of Series and Indexes. They have the same names as the regular Python string functions, but work on Series of strings.